**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/AndreaCimminoArriaga/LinkedData2021-2022/main/Assignment4/"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N96dea9128c994af2818fc4f0921fd4c8 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [ ]:
ns = Namespace("http://somewhere#")
for sub,pred,obj in g.triples((None, RDFS.subClassOf, ns.Person)):
    print(sub)

http://somewhere#Researcher


In [ ]:
from rdflib.plugins.sparql import prepareQuery


ns = Namespace("http://somewhere#")


q1 = prepareQuery('''
  PREFIX RDFS: <http://www.w3.org/2000/01/rdf-schema#>
  SELECT 
    ?Subclass
  WHERE { 
    ?Subclass RDFS:subClassOf ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)


for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [ ]:
ns = Namespace("http://somewhere#")
for sub,pred,obj in g.triples((None ,RDF.type, ns.Person)):
        print(sub)
for sub,pred,obj in g.triples((None, RDFS.subClassOf, ns.Person)):
    for sub,pred,obj in g.triples((None ,RDF.type, sub)):
        print(sub)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


In [ ]:
from rdflib.plugins.sparql import prepareQuery


ns = Namespace("http://somewhere#")


q1 = prepareQuery('''
  PREFIX RDFS: <http://www.w3.org/2000/01/rdf-schema#>
  SELECT 
    ?Persons
  WHERE { 
    {?Persons a ns:Person.} UNION {?class RDFS:subClassOf ns:Person. ?Persons a ?class}
  }
  ''',
  initNs = { "ns": ns}
)


for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [ ]:
ns = Namespace("http://somewhere#")
for individuals,pred,obj in g.triples((None, RDF.type, ns.Person)):
    for sub,properties,obj in g.triples((individuals, None, None)):
        for sub,obj,classes in g.triples((properties, RDF.type, None)):
            print(individuals,properties,classes)
for subclass,pred,obj in g.triples((None, RDFS.subClassOf, ns.Person)):
    for individuals,pred,obj in g.triples((None ,RDF.type, subclass)):
        for y,properties,obj in g.triples((individuals, None, None)):
            for sub,pred,classes in g.triples((properties, RDF.type, None)):
                print(individuals,properties,classes)


http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-

In [ ]:
from rdflib.plugins.sparql import prepareQuery


ns = Namespace("http://somewhere#")


q1 = prepareQuery('''
  PREFIX RDFS: <http://www.w3.org/2000/01/rdf-schema#>
  SELECT ?Persons ?properties ?class
  WHERE {
    {?individuals a ns:Person. ?individuals ?properties ?y.?properties a ?class.} UNION {?subclass RDFS:subClassOf ns:Person. ?individuals a ?subclass. ?individuals ?properties ?z. ?properties a ?class}
  }
  ''',
  initNs = { "ns": ns}
)


for r in g.query(q1):
  print(r)

(None, rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Datatype'))
(None, rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Datatype'))
(None, rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Property'))
(None, rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Datatype'))
(None, rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Datatype'))
(None, rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Property'))
(None, rdflib.term.URIRef('http://somewhere#http:/